In [17]:
import pandas as pd
import numpy as np
import os
import ast
import glob

#explode=False
#all_registers=False


In [18]:
#! cat HI_with_edu.jsonl | head -n 10 > test_HI_with_edu.jsonl

In [19]:
datas = []

labels_all_hierarchy_with_other = {
    "MT": ["MT"],
    "LY": ["LY"],
    "SP": ["SP", "it", "os"],
    "ID": ["ID"],
    "NA": ["NA", "ne", "sr", "nb", "on"],
    "HI": ["HI", "re", "oh"],
    "IN": ["IN", "en", "ra", "dtp", "fi", "lt", "oi"],
    "OP": ["OP", "rv", "ob", "rs", "av", "oo"],
    "IP": ["IP", "ds", "ed", "oe"],
}

final_scores = {"MT": 0.374349,
    "LY": 0.357626,
    "SP": 0.422314,
    "ID": 0.430686,
    "NA": 0.441469,
    "HI": 0.446558,
    "IN": 0.437347,
    "OP":  0.446629,
    "IP": 0.393026,
    "HI-IN":0.464502,
    "ne": 0.417933,
    "dtp": 0.452347,
    }

with os.scandir("result") as f:
    for file in f:
        filename = file.name
        print(f"reading {filename}")
        with open("result/"+filename) as f:
            data = [ast.literal_eval(line) for line in f]
        df_ = pd.DataFrame(data)
        df_ = df_.drop(columns="text")
        df_["parsed_register"] = df_["register"].map(lambda reg: [r for r in reg if r in list(labels_all_hierarchy_with_other.keys())+["ne", "dtp"]])
        df_["parsed_register"] = df_["parsed_register"].map(lambda reg: "-".join(reg))
        df_["parsed_register"] = df_["parsed_register"].map(lambda reg: {"NA-ne":"ne", "ne-NA": "ne", "IN-dtp":"dtp", "dtp-IN":"dtp", "IN-HI":"HI-IN"}.get(reg, reg))
        df_["final_score"] = df_["parsed_register"].map(lambda reg: final_scores[reg])
        datas.append(df_)



df = pd.concat(datas)

reading SP_with_edu.jsonl
reading MT_with_edu.jsonl
reading NA_with_edu.jsonl
reading dtp_with_edu.jsonl
reading LY_with_edu.jsonl
reading ID_with_edu.jsonl
reading ne_with_edu.jsonl
reading HI-IN_with_edu.jsonl
reading OP_with_edu.jsonl
reading IN_with_edu.jsonl
reading IP_with_edu.jsonl
reading HI_with_edu.jsonl


In [20]:
display(df)
print(np.unique(df["parsed_register"].tolist(), return_counts=True))

,id,register,score,int_score,parsed_register,final_score
0,fda1d9e32ccddfeccce6736a3262335f,[SP],1.273595,1,SP,0.422314
1,9048cdc5930abbbb6ed34cda3b0c0826,"[SP, it]",0.589984,1,SP,0.422314
2,674f705c1d1298b4b0ab40e549ba8004,[SP],0.896947,1,SP,0.422314
3,1c005ac1dad76eb176e683249b3b8608,"[it, SP]",0.850261,1,SP,0.422314
4,a415a49cd41df6f395075fdfebedd023,"[it, SP]",0.872051,1,SP,0.422314
...,...,...,...,...,...,...
9995,066478d3d7a4eac6801e54c75d295554,[HI],1.878192,2,HI,0.446558
9996,5d3f5e0859e4e2c05f6516ea3ebcb3fa,[HI],2.124670,2,HI,0.446558
9997,fc25e0a8e7ac20d2e7859e4af4292da5,[HI],0.587276,1,HI,0.446558
9998,e8aca2dc1c11b193607e143a09ad81cf,"[re, HI]",1.096759,1,HI,0.446558


(array(['HI', 'HI-IN', 'ID', 'IN', 'IP', 'LY', 'MT', 'NA', 'OP', 'SP',
       'dtp', 'ne'], dtype='<U5'), array([10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000, 10000,
       10000, 10000, 10000]))


In [21]:
def cooccurrence_matrix(df):
    regs = np.unique(df["parsed_register"]).tolist()
    gens = np.unique(df["int_score"]).tolist()
    print("regs: ", regs)
    print("gens: ", gens)

    print(range(len(regs)))
    reg2index = {k:v for k,v in zip(regs, range(len(regs)))}
    gen2index = {k:v for k,v in zip(gens, range(len(gens)))}

    m = np.zeros((len(regs), len(gens)), dtype=np.float32)
    summa = 0
    for i,d in df.iterrows():
        r = d["parsed_register"]
        g = d["int_score"]

        m[reg2index[r],gen2index[g]] += 1
        summa+=1

    return m/summa


m = cooccurrence_matrix(df)

assert 0.999 < sum(m.sum(axis=0)) < 1.001 and 0.999 < sum(m.sum(axis=1)) < 1.001

regs:  ['HI', 'HI-IN', 'ID', 'IN', 'IP', 'LY', 'MT', 'NA', 'OP', 'SP', 'dtp', 'ne']
gens:  [0, 1, 2, 3, 4, 5]
range(0, 12)


In [22]:
print(sum(m.sum(axis=0)))
print(sum(m.sum(axis=1)))

0.9999999999999998
1.0


In [23]:
def entropy(m):
    p_gen = m.sum(axis=0)
    p_reg = m.sum(axis=1)

    def _entropy_1d(v):
        return -1*sum(v*np.log2(v))

    def _entropy_2d(m):
        ent = 0
        for j in m:
            for k in j:
                if k != 0:
                    ent += k*np.log2(k)
        return -1*ent

    ent_reg = _entropy_1d(p_reg)
    ent_gen = _entropy_1d(p_gen)
    ent_reg_gen = _entropy_2d(m)

    return ent_reg, ent_gen, ent_reg_gen


def mutual_info(m):
    reg_pdf = m.sum(axis=1)
    gen_pdf = m.sum(axis=0)

    info = 0
    for i in range(len(reg_pdf)):
        for j in range(len(gen_pdf)):
            if m[i,j]!= 0:
                info += m[i,j]*np.log(m[i,j]/(reg_pdf[i]*gen_pdf[j]))
    return info


print(entropy(m))
print(mutual_info(m))

(3.584962500721156, 1.5292790225304054, 4.920277822321098)
0.13444539243092357


In [24]:
from scipy.stats import chi2_contingency

def cramers_v(df):
    x = df["parsed_register"]
    y = df["int_score"]

    confusion_matrix = pd.crosstab(x, y)
    print(confusion_matrix)
    chi2 = chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    return np.sqrt(phi2corr / min((kcorr-1), (rcorr-1)))
print(cramers_v(df))

int_score           0     1     2     3    4  5
parsed_register                                
HI                580  6996  2079   318   27  0
HI-IN             942  4324  3469  1057  205  3
ID               3537  5583   776   100    4  0
IN               1619  4829  2479   933  137  3
IP               5314  4499   168    17    2  0
LY               1351  7902   731    16    0  0
MT               5653  3962   367    17    1  0
NA               2563  6928   465    39    5  0
OP               1464  6241  2072   220    3  0
SP                287  7813  1746   145    9  0
dtp              1544  5742  1981   637   96  0
ne                791  6639  2281   261   28  0
0.23524780852696212


In [25]:
! pip install statsmodels

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Assuming 'nominal_var' is your nominal column and 'ordinal_var' is numeric (1 to 5)
model = ols('int_score ~ C(parsed_register)', data=df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
eta_squared = anova_table['sum_sq']['C(parsed_register)'] / anova_table['sum_sq'].sum()
print(eta_squared)

0.18648033351588342


In [ ]:
! pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

# Encode feature1
df['feature1_encoded'] = le.fit_transform(df['parsed_register'])

# Encode feature2
df['feature2_encoded'] = le.fit_transform(df['int_score'])

# Extract the encoded features
X = df[['feature1_encoded']]
y = df['feature2_encoded']

# Calculate mutual information
mi = mutual_info_classif(X, y, discrete_features=True)

print(f'Mutual Information between feature1 and feature2: {mi[0]}')

Mutual Information between feature1 and feature2: 0.1344453924309241


In [ ]:
! pip install plotly
! pip install nbformat>=4.2.0

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Example: sample data
# df = pd.DataFrame({
#     'parsed_register': ['A', 'B', 'A', 'C', 'B', 'C', 'A', 'B', 'C', 'A'],
#     'int_score': [1, 2, 2, 3, 3, 4, 1, 5, 4, 2]
# })

# Count pairs (grouped by nominal and ordinal labels)
df['int_score'] = df['int_score'].astype(str)  # make sure it's a string for plotting
counts = df.groupby(['parsed_register', 'int_score']).size().reset_index(name='count')

# Get all unique labels
all_labels = list(pd.unique(counts['parsed_register'].tolist() + counts['int_score'].tolist()))
label_to_index = {label: i for i, label in enumerate(all_labels)}

# Build Sankey inputs
source_indices = counts['parsed_register'].map(label_to_index)
target_indices = counts['int_score'].map(label_to_index)
values = counts['count']

# Create Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=all_labels,
    ),
    link=dict(
        source=source_indices,
        target=target_indices,
        value=values
    )
)])

fig.update_layout(title_text="Sankey Diagram: Register vs EDU score", font_size=12, width=600)
fig.show()
